In [2]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img,ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D,MaxPool2D, Dense , Dropout, Flatten




In [42]:
file_location = r"C:\Users\wilsen\OneDrive\Desktop\image classification\flowers/"
count = 0
fetch_images = os.listdir(file_location)
for images_in_files in fetch_images:
    files = list(os.listdir(file_location + images_in_files))
    count += len(files)
    print(images_in_files + " have " + str(len(files)) + " total")
print("total_amount = " + str(count))

daisy have 764 total
dandelion have 1052 total
rose have 784 total
sunflower have 733 total
tulip have 984 total
total_amount = 4317
